# Linear transformations

---
> Erick Eduardo Aguilar Hernández:
> * mat.ErickAguilar@gmail.com.mx
> * isc.ErickAguilar@gmail.com

---

Let $V$, $W$ be a vector spaces over $\mathbb{R}$  with $dim(V)=p$, $dim(W)=m$. Let $T: V \subset \mathbb{R}^p$ $\rightarrow$ $W \subset \mathbb{R}^m$ be a linear transformation. Then there exists a unique $p \times m$ matrix $\mathbf{A}$ such that $T\left[v\right]=A\left[v\right]$ for every $v \in V$. 

In some types of analysis a linear transformation of the data matrix it's required, so that each observation should be transformed to another vector space, note that each row could be interpreted as a vector in $\mathbb{R}^p$, so the matrix expression asociated to the linear transformation must be expressed as:

$$
\begin{align*} 
\mathbf{Y}_{m\times n}  & = \mathbf{A}_{m\times p}[\mathbf{X}]'_{p\times n}  \\
\mathbf{Y'}_{n\times m} & = \mathbf{X}_{n\times p}[\mathbf{A}]'_{p\times m} \\
\mathbf{Z}_{n\times m}  & = \mathbf{X}_{n\times p}[\mathbf{B}]_{p\times m} \\
\end{align*}
$$
  
  
   
$$
\left[ \begin{array}{ccccc}
z_{1 1} & \cdots & z_{1 m} \\
\vdots  & \ddots & \vdots \\
z_{i 1} & \cdots & z_{i m} \\
\vdots  & \ddots & \vdots \\
z_{n 1} & \cdots & z_{n m} \end{array} \right]
= \left[ \begin{array}{ccccc}
x_{1 1} & \cdots & x_{1 j} & \cdots & x_{1 p} \\
\vdots  & \ddots & \vdots  & \ddots & \vdots \\
x_{i 1} & \cdots & x_{i j} & \cdots & x_{i p} \\
\vdots  & \ddots & \vdots  & \ddots & \vdots \\
x_{n 1} & \cdots & x_{n j} & \cdots & x_{n p} \end{array} \right]
\left[ \begin{array}{ccccc}
b_{1 1} & \cdots & b_{1 m} \\
\vdots  & \ddots & \vdots \\
b_{j 1} & \cdots & b_{i m} \\
\vdots  & \ddots & \vdots \\
b_{p 1} & \cdots & b_{p m} \end{array} \right]
$$

In terms of linear combinations:

$$
\begin{align*}
z_{1 k}   & = b_{1 k} x_{1 1} + \cdots b_{j k} x_{1 j} + \cdots + b_{p k} x_{1 p}   \\
\vdots    & = \vdots \\
z_{n k}   & = b_{1 k} x_{1 1} + \cdots b_{j k} x_{1 j} + \cdots + b_{p k} x_{1 p} \\
\end{align*}
$$